In [31]:
import math
from tkinter import *
from tkinter.ttk import *
from tkinter.filedialog import askopenfile
import tkinter as tk
from PIL import Image,ImageTk
from decimal import *

class Automat:
    
    def __init__(self, filename,canva_erreur):
        
        self.not_found = False
        try:
            # öffnen die Datei
            file = open(filename)
        except:
            lbl1 = tk.Label(
            canva_erreur, 
            text="Datei nicht gefunden, versuchen Sie es erneut",
            fg="red"
                                        )   
            lbl1.place(x=0,y=0)
            self.not_found = True
        else:
            self.not_found = False
        
        if not self.not_found:
            # Lesen den Inhalt der geöffneten Datei
            self.content = file.readlines()
        
            #Informationen
            self.__information = self.content[0:3]
        
            try:
                #Anfangsbestand erhalten
                temp = self.content[3].split(";")
                self.__stock =[int(temp[i]) for i in range(len(temp))]
                self.__stock.insert(0,0)
                self.__stock.insert(0,0)
            except:
                lbl1 = tk.Label(
                canva_erreur, 
                text="Es ist nicht die Form einer Eingabedatei",
                fg="red"
                                        )   
                lbl1.place(x=0,y=0)
                self.not_found = True
            
            else:
        
                #Zahl von Geschäftsvorgängen
                self.__transaction_len = len(self.content) - 4
        
                #Geschäftsvorgänge
                self.__transaction = list()
                for i in range(4,len(self.content)):
                    self.__transaction.append(self.content[i])
            
                #die eingegangenen Beträge (Müntzen)
                self.__received_coins = [0,0,0,0,0]
    
                #die eingegangenen Beträge (Scheine)
                self.__received_bills = [0,0]
        
        
    
    
    """
    Das ist getter von stock
    """
    def get_stock(self):
        return self.__stock
    
    
    
    """
    Diese Funktion Schreibt die Informationen
    von der Eingabdatei in die Ausgabedatei
    """
    def __write_information(self,f,window):
        f.write(self.__information[0])
        f.write(self.__information[1])
        f.write(self.__information[2])
        lbl1 = tk.Label(
                        window, 
                        text= self.__information[0],
                        font=("Arial", 13)
                                    )
        lbl1.place(x=10, y=10)
        lbl2 = tk.Label(
                        window, 
                        text= self.__information[1],
                        font=("Arial", 13)
                                    )
        lbl2.place(x=10, y=40)
        lbl3 = tk.Label(
                        window, 
                        text= self.__information[2],
                        font=("Arial", 13)
                                    )
        lbl3.place(x=10, y=70)
    
    
   
    """
    Diese Funktion ziegt der aktuell Stock
    von dem Automat
    """
    def consult_stock(self):
        coins = ["10€","5€","2€","1€","50 Cent","20 Cent","10 Cent"]
        stock = self.__stock
        for i in range(len(stock)):
            print("Zahl von ",coins[i]," : ",stock[i])
            
    
    
    """
    Diese Funktion berechnet das Wechselgeld
    von jede Transaktion
    """
    def changes_calculation(self, price):
        changes = [0,0,0,0,0]
        
        while not math.isclose(price, 0, rel_tol=1e-5):
            if (price >= 2 or math.isclose(price, 2, rel_tol=1e-5)) and self.__stock[2]-changes[0] > 0:
                price -= 2.0 
                changes[0] += 1
            elif (price >=1 or math.isclose(price, 1, rel_tol=1e-5)) and self.__stock[3]-changes[1] > 0:
                price -= 1.0
                changes[1] += 1
            elif (price >=0.5 or math.isclose(price, .5, rel_tol=1e-5)) and self.__stock[4]-changes[2] > 0:
                price -= 0.5
                changes[2] += 1
            elif (price >=0.2 or math.isclose(price, .2, rel_tol=1e-5)) and self.__stock[5]-changes[3] > 0:
                price -= 0.2
                changes[3] += 1
            elif (price >=0.1 or math.isclose(price, .1, rel_tol=1e-5)) and self.__stock[6]-changes[4] > 0:
                price -= 0.1
                changes[4] += 1
            elif price < 0.01:
                price = 0
            else:
                print("Wächselgeld ist unzureichend")
                return None
        return changes
                            
    
    
    """
    Diese Funktion berechent die Menge
    von Müntzen bzw. Scheine erhalten
    in jeder Transaktion
    """
    def count_received_pieces(self, transaction):
        received_money = [0,0,0,0,0,0,0]
        for j in range(1,len(transaction)):
            
            num_piece = int(transaction[j])
            
            received_money[j-1] += num_piece
            
            if j <= 2 and num_piece != 0:
                self.__received_bills[j-1] += num_piece
                        
            elif j > 2 and num_piece != 0:
                self.__received_coins[j-3] += num_piece
        return received_money
    
    
    
    """
    Diese Funktion berechnet die Summe 
    von Geld aus einer Liste
    """
    def get_sum_of_money(self,lst):
        value = [10.00,5.00,2.00,1.00,0.50,0.20,0.10]
        amount = 0
        for i in range(len(lst)):
            amount = Decimal(amount) + (Decimal(float(lst[i])) * Decimal(value[i]))
    
        return amount
    
    
    
    """
    Diese Funktion wandelt eine 
    Liste in eine Zeichenfolge um
    """
    def list_to_str(self,lst):
        mystr = ""
        for i in lst:
            mystr += str(i)+";"
        return mystr[:len(mystr)-1]
    
    
    
    """
    Diese Funktion schafft eine neue Datei 
    (wenn es schön keine Datei mit dem gleinchen Namen gibt ) 
    und Schreibt sie seine Header
    """
    def get_writing_file(self,filename,window):
        f = open(filename, "w",encoding='utf-8')
        f.seek(0)
        f.truncate()
        self.__write_information(f,window)
        initial_money = self.float_to_string(self.get_sum_of_money(self.__stock))
        initial_stock = self.list_to_str(self.__stock)
        f.write("Anfangsbestand: "+initial_money+"€ ("+initial_stock+")\n")
        return f
    
    
    
    """
    Diese Funktion sammelt alle Berechnungen,
    erstellt eine neue Datei ,indem sie alle 
    Ergebnisse der Berechnungen speichert
    """
    def do_transactions(self,ausgabedatei_filename,ws,window):
        coins = [10,5,2,1,0.5,0.20,0.10]
        changes = list()
        file = self.get_writing_file(ausgabedatei_filename,window)
        line_count=1
        entred_pieces = [0]*7
        x0 = 10
        for i in self.__transaction:
            received_money = 0
            transaction = i.split(";")
            try:
                price = float(transaction[0].replace(",","."))
                number_dec_str = str(price-int(price))
                if len(number_dec_str) > 3:
                    number_dec = int(number_dec_str[3:4])
                else:
                    number_dec = 0
            
                for j in range(1,len(transaction)):
                    
                    #Erhalten Geld berechnen
                    num_piece = int(transaction[j])
                    received_money += num_piece * coins[j-1]
                    entred_pieces[j-1] = num_piece
                 
                received_money_str = self.float_to_string(received_money)
                
                
                if received_money >= price and price >= 0.9 and price <= 19.9 and number_dec == 0:
                    changes = self.changes_calculation(received_money - price)
                    
                    if changes is None:
                        
                        print("unmögliche Transaktion nehmen Sie Ihr Geld zurück")
                        file.write(str(line_count)+". Wächselgeld sind unzureichend, "+"unmögliche Transaktion nehmen Sie Ihr Geld zurück\n")
                        lbl = tk.Label(ws, 
                           text="                    "+str(line_count)+". Buchung: \nEingefügtes Geld: "+received_money_str+"€ / Preis: "+str(price)+"€",
                                        )
                        lbl.place(x=x0, y=130)
                        lbl2 = tk.Label(
                                    ws, 
                                    text="Wächselgeld sind unzureichend, \n"+"unmögliche Transaktion nehmen Sie\n Ihr Geld zurück\n",
                                    font=("Arial", 15),  
                                    fg="red"
                                    )
                        lbl2.place(x=x0, y=200)
                        
                        line_count +=1
                        x0 += 440
                        continue
                        
                    else:
                        changes.insert(0,0)
                        changes.insert(0,0)
                        
                        #Zahl von Müntzen bzw. Scheinen speichern
                        received_coins = self.count_received_pieces(transaction)
                        self.__stock = [self.__stock[k] + received_coins[k] for k in range(len(received_coins))]
                        self.__stock = [self.__stock[k] - changes[k] for k in range(len(changes))]
                        
                        
                        gesamtband = self.float_to_string(self.get_sum_of_money(self.__stock))
                            
                       
                        self.write_in_file(file,line_count,received_money_str,received_money,price,changes,gesamtband)
                        
                        self.insert_transaction_result(ws,x0,price,received_money_str,received_money,entred_pieces,changes,gesamtband,line_count)
                        
                else:
                    
                    if received_money < price:
                        file.write(str(line_count)+". Sie haben nicht genug Geld eingefügt, "+"unmögliche Transaktion nehmen Sie \n Ihr Geld zurück\n")
                        lbl2 = tk.Label(
                                    ws, 
                                    text=str(line_count)+". Sie haben nicht genug Geld eingefügt\n, "+"unmögliche Transaktion nehmen Sie \n Ihr Geld zurück\n",
                                    font=("Arial", 15),  
                                    fg="red"
                        )
                        lbl2.place(x=x0, y=120)
                    if price < 0.9 :
                        file.write(str(line_count)+". invalide buchung, "+"der Preis ist \nweniger als 0.9 euro\n")
                        lbl2 = tk.Label(
                                    ws, 
                                    text=str(line_count)+". invalide buchung, "+"der Preis ist \nweniger als 0.9 euro\n",
                                    font=("Arial", 15),  
                                    fg="red"
                        )
                        lbl2.place(x=x0, y=120)
                    if price > 19.9:
                        file.write(str(line_count)+". invalide buchung, "+"der Preis ist \nüber als 19.9 euro\n")
                        lbl2 = tk.Label(
                                    ws, 
                                    text=str(line_count)+". invalide buchung, "+"der Preis ist \nüber als 19.9 euro\n",
                                    font=("Arial", 15),  
                                    fg="red"
                        )
                        lbl2.place(x=x0, y=120) 
                    if number_dec != 0:
                        file.write(str(line_count)+". invalide buchung, "+"Invalid Preis\n")
                        lbl2 = tk.Label(
                                    ws, 
                                    text=str(line_count)+". invalide buchung, "+"Invalid Preis\n",
                                    font=("Arial", 15),  
                                    fg="red"
                        )
                        lbl2.place(x=x0, y=120) 
                    
                line_count+=1
                x0 += 440
            except ValueError as verr:
                print("Die Angegebene Wert ist kein Zahl")


    def insert_coins(self,x0 ,y0, all=1):
        if all == 1:
            coins=["10_euro","5_euro","2_euro","1_euro","50_cent","20_cent","10_cent"]
        else:
            coins=["2_euro","1_euro","50_cent","20_cent","10_cent"]
        emp = x0
        for i in coins:
            # Create a photoimage object of the image in the path
            image1 = Image.open("data/"+str(i)+".png").resize((40, 40))
            test = ImageTk.PhotoImage(image1)

            label1 = tk.Label(image=test)
            label1.image = test

            # Position image
            label1.place(x=emp, y=y0)
            emp += 50 
            
        
    def insert_amount_coins(self,x0,y0,lst):
        emp = x0
        for i in lst:
            lbl = tk.Label(
            ws, 
            text=str(i),
            )
            lbl.place(x=emp, y=y0)
            emp += 50
    
    def insert_anfangsbestand(self,ws):
        initial_stock = self.__stock
        anfangsbestand = str(self.get_sum_of_money(initial_stock))
        if len(anfangsbestand)>4:
            anfangsbestand=anfangsbestand[:5]
        lbl = tk.Label(
        ws, 
        text='Anfangsbestand:'+anfangsbestand+'€',
        )
        lbl.place(x=450, y=20)
    
        self.insert_coins(450,220)
        self.insert_amount_coins(465,270,initial_stock)
        
        
        
    
    def insert_transaction_result(self,ws,x0,price,received_money_str,received_money,entred_pieces,changes,gesamtband,line_count):
        lbl = tk.Label(ws, 
                       text="                    "+str(line_count)+". Buchung: \nEingefügtes Geld: "+received_money_str+"€ / Preis: "+str(price)+"€",
                                        )
        lbl.place(x=x0, y=130)
        self.insert_coins(x0,352)
        self.insert_amount_coins(x0+15,405,entred_pieces)
        lbl2 = tk.Label(
        ws, 
        text="Rückgabe: "+str(Decimal(received_money) - Decimal(price))+"€",
                                        )
        lbl2.place(x=x0, y=250)
        self.insert_coins(x0,455,0)
        self.insert_amount_coins(x0+15,505,changes[2:])
        lbl1 = tk.Label(
        ws, 
        text="Gesamtbestand "+gesamtband+"€",
                                        )   
        lbl1.place(x=x0,y=350)
        self.insert_coins(x0,550)
        self.insert_amount_coins(x0+15,597,self.__stock)
    
    
    def write_in_file(self,file,line_count,received_money_str,received_money,price,changes,gesamtband):
        
        line = str(line_count)+". Buchung: Eingefügtes Geld: "+received_money_str+"€ / Preis: "+str(price)+"€ / Rückgabe: "+str(received_money - price)+"€ ("+self.list_to_str(changes[2:])+")"        
        line += " / Gesamtbestand "+gesamtband        
        line += "€ ("+self.list_to_str(self.__stock)+")\n"
        
        file.write(line)
        
    
    def float_to_string(self,myfloat):
        myfloat_str = str(myfloat)
        if len(myfloat_str)>4:
            myfloat_str = myfloat_str[:5]
        return myfloat_str

In [32]:
ws = Tk()
ws.title('Automat System')
ws.geometry('1300x650') 
canva = Canvas(ws, width=1300, height=450,relief=SUNKEN )
canva.place(x=0,y=180)
canva_info = Canvas(ws, width=450, height=90,relief=SUNKEN)
canva_info.place(x=20,y=70)
getcontext().prec = 3

def open_file():
   
    file_path = askopenfile(mode='r', filetypes=[('Text Files', '*txt')])
        
    if file_path is not None:
        pass
    input_file_label.config(text = file_path.name)


def start(canva,canva_info):
    canva_erreur = Canvas(ws, width=250, height=20,relief=SUNKEN)
    canva_erreur.place(x=520,y=117)
    canva_info.destroy()
    canva_info = Canvas(ws, width=450, height=90,relief=SUNKEN)
    canva_info.place(x=20,y=70)
    
    canva.destroy()
    canva = Canvas(ws, width=1300, height=450,relief=SUNKEN )
    canva.place(x=0,y=180)
    filename = input_file_label.cget("text")
    
    automat = Automat(filename,canva_erreur)   
    stock = automat.get_stock()
    pos = True
    for i in stock:
        if i<0:
            pos = False
    
    if pos :
        if not automat.not_found:
            automat.insert_anfangsbestand(canva)
    
            automat.do_transactions("Ausgabedatei.txt",canva,canva_info)
    else:
        lbl1 = tk.Label(
            canva_erreur, 
            text="Das Bestand ist negativ",
            fg="red"
                                        )   
        lbl1.place(x=5,y=0)
    
    
w = Canvas(ws, width=1300, height=60)
w.config(bg='black') 
w.place(x=0,y=0)
l = tk.Label(ws, text = "Automat System",fg="white",bg="black")
l.config(font =("Helvetica", 24,"bold"))
l.place(x=525, y=10)


line = Canvas(ws, width=1300, height=5)
line.config(bg='black') 
line.place(x=0,y=170)


y = Canvas(ws, width=200, height=22,relief=RAISED )
y.config(bg='white') 
y.place(x=515,y=90)
input_file_label = tk.Label(
    ws, 
    text='Eingabedatei hochladen',
    bg="white"
    )
input_file_label.place(x=520,y=92)



input_file_btn = Button(
    ws, 
    text ='Datei wählen', 
    command = lambda:open_file()
    ) 
input_file_btn.place(x=700,y=89.5)



upld = Button(
    ws, 
    text='Transaktion durchführen', 
    command= lambda:start(canva,canva_info)
    )
upld.place(x=580,y=140)


ws.mainloop()

0 * 10.0 = 0
0 * 5.0 = 0
10 * 2.0 = 20
10 * 1.0 = 30
10 * 0.5 = 35.0
10 * 0.2 = 37.0
10 * 0.1 = 38.0
0 * 10.0 = 0
0 * 5.0 = 0
10 * 2.0 = 20
10 * 1.0 = 30
10 * 0.5 = 35.0
10 * 0.2 = 37.0
10 * 0.1 = 38.0
0 * 10.0 = 0
1 * 5.0 = 5
10 * 2.0 = 25
10 * 1.0 = 35
10 * 0.5 = 40.0
9 * 0.2 = 41.8
9 * 0.1 = 42.7
1 * 10.0 = 10
1 * 5.0 = 15
10 * 2.0 = 35
10 * 1.0 = 45
10 * 0.5 = 50.0
9 * 0.2 = 51.8
9 * 0.1 = 52.7
1 * 10.0 = 10
1 * 5.0 = 15
12 * 2.0 = 39
10 * 1.0 = 49
10 * 0.5 = 54.0
7 * 0.2 = 55.4
9 * 0.1 = 56.3
